# 环境准备

In [ ]:
!git clone https://github.com/mymusise/ChatGLM-Tuning.git

In [2]:
%cd ChatGLM-Tuning

/output/ChatGLM-Tuning


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://github.com/huggingface/peft.git (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /tmp/pip-req-build-2vf9v8o2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-2vf9v8o2
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running command git fetch -q https://github.com/huggingface/peft.git e536616888d51b453ed354a6f1e243fecb02ea08
  Running command git checkout -q e536616888d51b453ed354a6f1e243fecb02ea08
  Resolved https://github.com/huggingface/peft.git to commit e536616888d51b453ed354a6f1e243fecb02ea08
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 45.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 71.8 MB/s eta 0

# 准备数据
## cover_alpaca2jsonl.py

In [4]:
import json
from tqdm import tqdm


def format_example(example: dict) -> dict:
    context = f"Instruction: create userstory tasks \n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}


def main():
    with open("/openbayes/input/input0/datasets/userstory_map.jsonl") as f:
        examples = list(f)

    with open("/output/train.jsonl", 'w') as f:
        for example in tqdm(examples, desc="formatting.."):
            f.write(json.dumps(format_example(json.loads(example))) + '\n')


if __name__ == "__main__":
    main()

formatting..: 100%|██████████| 186/186 [00:00<00:00, 36562.80it/s]


## tokenize_dataset_rows.py

In [5]:
import json
from tqdm import tqdm

import datasets
import transformers


def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True,return_attention_mask=False,
                add_special_tokens=False)
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        return_attention_mask=False,
        add_special_tokens=False)
    input_ids = prompt_ids + [150001, 150004] + target_ids + [150005]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}


def read_jsonl(path, max_seq_length, skip_overlength=False):
    model_name = "/openbayes/input/input1"
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = transformers.AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature


def main():
    dataset = datasets.Dataset.from_generator(
        lambda: read_jsonl("/output/train.jsonl", 384, False)
    )
    dataset.save_to_disk("/output/train")


if __name__ == "__main__":
    main()

Generating train split: 0 examples [00:00, ? examples/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.

100%|██████████| 186/186 [00:00<00:00, 1445.77it/s]


Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-5161fd2f24be51ae/0.0.0. Subsequent calls will reuse this data.


Saving the dataset (0/1 shards):   0%|          | 0/186 [00:00<?, ? examples/s]

# 训练

## 训练前推理

In [5]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = AutoModel.from_pretrained("/openbayes/input/input1", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//10.111.0.1'), PosixPath('443'), PosixPath('tcp')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//alchemist-experience'), PosixPath('7890'), PosixPath('http')}
  warn(msg)
/usr/loc


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [6]:
def format_example(example: dict) -> dict:
    context = f"Instruction: create userstory tasks \n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

In [ ]:
import json

with open("/openbayes/input/input0/datasets/userstory_map.jsonl") as f:
    examples = list(f)

with torch.no_grad():
    idx = 0
    for example in examples[:5]:
        item = json.loads(example)
        feature = format_example(item)
        input_text = feature['context']
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        inputs = model.prepare_inputs_for_generation(input_ids)
        for k,v in inputs.items():
            if v is not None:
                inputs[k] = v.to("cuda")
        outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
        out = outputs[0].tolist()[input_ids.size()[-1]:]
        answer = tokenizer.decode(out)
        item['infer_answer'] = answer
        print(input_text)
        print(answer)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        idx += 1

## 训练

In [7]:
!sed -i "s/THUDM\/chatglm-6b/\/openbayes\/input\/input1/" finetune.py
!cat finetune.py
!python finetune.py \
    --dataset_path /output/train \
    --lora_rank 8 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 1 \
    --max_steps 3000 \
    --save_steps 1000 \
    --save_total_limit 2 \
    --learning_rate 1e-4 \
    --fp16 \
    --remove_unused_columns false \
    --logging_steps 50 \
    --output_dir /output/lora

from transformers.integrations import TensorBoardCallback
from torch.utils.tensorboard import SummaryWriter
from transformers import TrainingArguments
from transformers import Trainer, HfArgumentParser
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType
from dataclasses import dataclass, field
import datasets
import os


tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)


@dataclass
class FinetuneArguments:
    dataset_path: str = field(default="data/alpaca")
    model_path: str = field(default="output")
    lora_rank: int = field(default=8)


class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature i

# 训练后推理

In [9]:
import torch

from transformers import AutoModel
from transformers import AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained("/openbayes/input/input1", trust_remote_code=True, device_map='auto')

tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)

peft_path = "/output/lora/adapter_model.bin"
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)

model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/usr/local/lib/python3.8/site-packages/peft/tuners/lora.py:191: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


## 原样输入

In [11]:
import json

def format_example(example: dict) -> dict:
    context = f"Instruction: create userstory tasks \n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

with open("/openbayes/input/input0/datasets/userstory_map.jsonl") as f:
    examples = list(f)

with torch.no_grad():
    idx = 0
    for example in examples[:5]:
        item = json.loads(example)
        feature = format_example(item)
        input_text = feature['context']
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        inputs = model.prepare_inputs_for_generation(input_ids)
        for k,v in inputs.items():
            if v is not None:
                inputs[k] = v.to("cuda")
        outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
        out = outputs[0].tolist()[input_ids.size()[-1]:]
        answer = tokenizer.decode(out)
        item['infer_answer'] = answer
        print(input_text)
        print(answer)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        idx += 1

Instruction: create userstory tasks 
Input:  Animation and Comics
Answer: 

User Tasks:
1. Browse and search for animations and comics
2. View details of animations and comics
3. Create an account
4. Log in to the account
5. Add animations and comics to favorites
6. Download animations and comics
7. Share animations and comics with friends
8. Rate and review animations and comics
9. Create and upload animations and comics
10. Participate in online forums and discussions
### 1.Answer:
 
User Tasks:
1. Browse and search for animations and comics
2. View details of animations and comics
3. Create an account
4. Log in to the account
5. Add animations and comics to favorites
6. Download animations and comics
7. Share animations and comics with friends
8. Rate and review animations and comics
9. Create and upload animations and comics
10. Participate in online forums and discussions 


Instruction: create userstory tasks 
Input:  Business Services
Answer: 

User Tasks:
1. Log in to the Busin

## 任意Prompt推理

In [14]:
def evaluate(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    inputs = model.prepare_inputs_for_generation(input_ids)
    for k,v in inputs.items():
        if v is not None:
            inputs[k] = v.to("cuda")
    outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
    out = outputs[0].tolist()[input_ids.size()[-1]:]
    answer = tokenizer.decode(out)

    print(input_text)
    print(answer)

In [15]:
evaluate("Instruction: create userstory tasks \nInput: 淘宝\nAnswer:")

Instruction: create userstory tasks 
Input: 淘宝
Answer:

User Tasks:
1. Create an account
2. Log in to the application
3. View available products
4. Select a product to view
5. View product details
6. Purchase a product
7. Log out of the application


In [16]:
evaluate("Instruction: create userstory tasks \nInput: 电影网站\nAnswer:")

Instruction: create userstory tasks 
Input: 电影网站
Answer:

User Tasks:
1. Search for movies
2. View movie details
3. Add movies to favorites
4. View movie ratings and reviews
5. View movie schedules
6. View movie ratings and reviews
7. Share movies with friends
8. View movie trailers
9. View movie budgets
10. View movie theaters near me
11. View movie theaters list
12. View movie theaters near location
13. View movie tips and tricks
14. View movie tutorials
15. View movie trailers
16. View movie trailers
17. View movie trailers
18. View movie trailers
19. View movie trailers
20. View movie trailers
21. View movie trailers
22. View movie trailers
23. View movie trailers
24. View movie trailers
25. View movie trailers
26. View movie trailers
27. View movie trailers
28. View movie trailers
29. View movie trailers
30. View movie trailers


In [17]:
evaluate("Instruction: create userstory tasks \nInput: 团购网站\nAnswer:")

Instruction: create userstory tasks 
Input: 团购网站
Answer:

User Tasks:
1. Search for团购商品
2. View团购信息
3. Add团购商品 to cart
4. Make payment
5. Receive团购商品
6. View团购商品
7. Share团购商品
8. Check out团购商品
9. Receive团购 result
10. View团购信息
11. Make团购信息
12. Check团购信息
13. Check out团购信息
14. Make payment团购
15. View团购信息
16. View团购商品
17. View团购评论
18. Leave团购评论
19. Get团购 feedback
20. Get团购 update


In [18]:
evaluate("Instruction: create userstory tasks \nInput: 博客网站\nAnswer:")

Instruction: create userstory tasks 
Input: 博客网站
Answer:

User Tasks:
1. Search for博客网站
2. View博客网站
3. Follow other users
4. Create and save的博客 posts
5. View博客 post
6. Share博客 post
7. Write and save blog posts
8. View blog posts
9. Edit blog posts
10. Follow other bloggers
11. View bloggers' blog posts
12. View blog posts by other bloggers


In [19]:
evaluate("Instruction: create userstory tasks \nInput: 京西商城\nAnswer:")

Instruction: create userstory tasks 
Input: 京西商城
Answer:

User Tasks:
1. Search for products by category
2. View product details
3. Add products to cart
4. Checkout and pay for products
5. Receive order confirmation
6. Track order status
7. View order history
8. Leave feedback on product quality, service, or other issues
9. Contact customer service for assistance
10. Read product descriptions and reviews


In [ ]:
evaluate("Instruction: create userstory tasks \nInput: 大信聊天\nAnswer:")